In [17]:
import pandas as pd
import tiktoken
from langchain_community.document_loaders import DataFrameLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain.vectorstores import faiss
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore

In [2]:
df = pd.read_csv('./data/law-all-data.csv')
df = df.drop(['input'], axis = 1)
df

,instruction,output
0,동거 중인데 갑자기 헤어지자는 통보를 받았어요. 사실혼인 경우에도 위자료를 받을 수...,위자료를 받을 수 있습니다. 사실혼은 부부간 합의 또는 부부 어느 한 쪽의 일방적인...
1,이혼이 취소될 수 있나요?,"재판상 이혼은 재판절차를 거쳐 이혼판결이 선고된 것이므로 취소될 수 없지만, 협의이..."
2,이혼해도 자녀를 만날 수 있나요?,이혼 후 자녀를 직접 양육하지 않는 부모 일방과 자녀는 서로 만나거나 연락할 수 있...
3,이혼한 후에 자녀의 성과 본을 저의 성과 본으로 바꿀 수 있나요?,이혼 후 자녀의 성과 본을 자신의 성과 본으로 바꿀 수 있습니다. ◇ 법원 허가 자...
4,"중학생 딸아이가 학교 숙제로 유언장을 작성했는데, 이 유언장이 법적으로 효력 있는 ...",유언은 의사능력이 있는 17세(유언 적령)에 달한 사람이 할 수 있습니다. 따라서 ...
...,...,...
2270,"시청에 정기적으로 문구류를 납품하는 수의계약에 참여했는데, 수의계약의 계약상대자는 ...","수의계약대상자는 견적제출자의 견적가격과 계약이행능력 등에 따라 결정되며, 원칙적으로..."
2271,물품계약을 체결한 후 물가가 급격히 올라서 계약 당시의 금액으로는 수량을 맞추기 어...,아니요. 물품계약을 체결한 날부터 90일 이상 지난 후 입찰일을 기준일로 하여 품목...
2272,"계약을 체결한 후에는 계약 완료 전이라도 대금을 미리 받을 수 있다고 하던데, 얼마...",계약을 이행하기 전이라도 일정 요건에 해당하면 계약금액의 100분의 70을 초과하지...
2273,물품을 납품하기 직전에 천재지변 등 불가항력의 사유로 대형화재가 발생하여 해당 물품...,"계약상대자의 책임 없이 이행이 지체되는 경우에는 계약기간 연장신청을 할 수 있고, ..."


In [3]:
text_col = []
token_col = []
encoder = tiktoken.encoding_for_model("gpt-3.5-turbo")

for _, row in df.iterrows():
    instruction = str(row["instruction"])
    response = str(row["output"])
    
    text = "### Instruction:\n" + instruction + "\n### Response:\n" + response
    text_col.append(text)
    token = encoder.encode(text)
    token_col.append(len(token))

df.loc[:, "text"] = text_col
df.loc[:, "token"] = token_col
df.drop(['instruction', 'output'], axis=1, inplace=True)

In [15]:
# print(f'{max(df['token']), pd.Series(['token'], dtype="int"}').sum())
m = max(df['token'])
t = pd.Series(df['token']).sum()
print(f'Max Token: {m}\nTotal Token: {t}')

Max Token: 1101
Total Token: 1479566


In [18]:
loader = DataFrameLoader(df, page_content_column="text")

In [19]:
loader.load()

[Document(page_content='### Instruction:\n동거 중인데 갑자기 헤어지자는 통보를 받았어요. 사실혼인 경우에도 위자료를 받을 수 있나요?\n### Response:\n위자료를 받을 수 있습니다. 사실혼은 부부간 합의 또는 부부 어느 한 쪽의 일방적인 파기에 의해 해소될 수 있습니다. 이 때 정당한 사유(「민법」 제840조에 준하는 사유) 없이 일방적으로 사실혼을 파기한 배우자 또는 사실혼 파탄의 원인을 제공한 제3자(예를 들어, 배우자의 부모 등)에게 위자료를 청구할 수 있습니다. 위자료에 관한 합의가 이루어지지 않으면 법원에 위자료청구소송을 제기해서 위자료를 받을 수 있습니다. ◇ 「민법」 제840조에서 정한 이혼사유(재판상 이혼사유) 1. 배우자에게 부정한 행위가 있었을 때 2. 배우자가 악의로 다른 일방을 유기한 때 3. 배우자 또는 그 직계존속으로부터 심히 부당한 대우를 받았을 때 4. 자기의 직계존속이 배우자로부터 심히 부당한 대우를 받았을 때 5. 배우자의 생사가 3년 이상 분명하지 않은 때 6. 그 밖에 혼인을 계속하기 어려운 중대한 사유가 있을 때'),
 Document(page_content='### Instruction:\n이혼이 취소될 수 있나요?\n### Response:\n재판상 이혼은 재판절차를 거쳐 이혼판결이 선고된 것이므로 취소될 수 없지만, 협의이혼은 부부간 자유로운 의사에 따른 합의에 기초하므로 사기 또는 강박(强迫)으로 인해 이혼의 의사표시를 했다면 이혼이 취소될 수 있습니다. 이혼을 취소하려면 관할 가정법원에 이혼취소소송을 제기해야 합니다. 이혼취소판결이 확정되면 그 이혼은 처음부터 없었던 것과 같아지므로 취소판결 전에 다른 일방이 재혼했다면 그 재혼은 중혼(重婚)이 됩니다. ◇ 중혼 ☞ 중혼은 법률상 혼인관계가 둘 이상 존재하는 위법한 상태로서 혼인의 취소사유가 됩니다. 중혼을 이유로 재혼(後婚)이 취소되면 전 배우자와만 법률상의 부부관계가 유지되고, 재혼 배우자와의 법률상 부부관계는 종료됩니

In [19]:
model_name = "Salesforce/SFR-Embedding-Mistral"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

cache_dir = LocalFileStore("./.cache/")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
vectorstore = FAISS.from_documents(loader, cached_embeddings)

No sentence-transformers model found with name /Users/changhyunkim/.cache/torch/sentence_transformers/Salesforce_SFR-Embedding-Mistral. Creating a new one with MEAN pooling.


: 

: 